# 🎯 Part 1: Building Your First AI Agent
## Content Creation Studio Workshop

Welcome to the first part of our series on Google's Agent Development Kit (ADK)!
In this series, we'll go from the absolute basics to building complex, multi-agent
content creation systems.

---

## What is Google ADK?

According to the official documentation, the **Agent Development Kit (ADK)** is a framework
designed to help developers build sophisticated, autonomous AI systems. It's not just
about creating a simple chatbot; ADK provides the building blocks — like agents, tools,
sessions, and runners — to construct complex workflows where multiple AI agents can
collaborate to solve problems.

Today, we'll start with the most fundamental step: building and running a single,
simple agent that helps **generate blog topic ideas**.

📖 **Documentation**: [Google ADK Overview](https://google.github.io/adk-docs/)

---

## 📚 The Content Creation Studio Playbook Series
- **Part 1: Building Your First AI Agent** (You are here) 🎯
- Part 2: Giving Your Agent Custom Tools
- Part 3: Building Agent Teams with Agent-as-a-Tool
- Part 4: Multi-Step Workflows with SequentialAgent
- Part 5: Building Iterative Workflows with LoopAgent
- Part 6: Efficient Workflows with ParallelAgent
- Part 7: The Capstone Project

---

## 🎓 New Concepts in This Part

In Part 1, we'll introduce these **fundamental ADK concepts**:

1. **Agent** - A self-contained execution unit designed to act autonomously
2. **Tools** - Built-in capabilities agents can use (like Google Search)
3. **Session** - Memory of a conversation containing message history
4. **SessionService** - Manages session lifecycle (create, retrieve, update)
5. **Runner** - The execution engine that orchestrates agent invocations

Each concept will be clearly marked with 🆕 when first introduced!

## ⚙️ 1. Setup: Install Libraries

First, let's install the specific version of the Google Agent Development Kit (ADK) that this notebook is built with. Pinning the version ensures our code will always work as expected.

📖 **Documentation**: [ADK Installation Guide](https://google.github.io/adk-docs/get-started/)

In [ ]:
!pip install google-adk==1.19.0 -q

## 🔑 2. Authentication: Configure Your API Key

Next, we need to securely provide our Google API key. This code will create a secure input prompt for you to paste your key. It then sets the key as an environment variable, which is the standard way the ADK authenticates your requests.

### How to Get Your API Key:
1. Navigate to [Google AI Studio](https://aistudio.google.com)
2. Sign in with your Google Account
3. Click the "Get API key" button
4. Follow the prompts to select or create a Google Cloud project
5. Copy your API key and paste it below when prompted

**Note:** The Gemini API has a free-of-charge tier. Following the steps in this beginner's tutorial will not cost you any money!

In [ ]:
import os
from getpass import getpass

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

## 🤖 3. Define Your First Agent

### 🆕 NEW CONCEPT: Agent

> **What is an Agent?**  
> An agent is "a self-contained execution unit designed to act autonomously to achieve specific goals." Agents can perform tasks, interact with users, utilize external tools, and coordinate with other agents.
>
> 📖 **Reference**: [ADK Agents Documentation](https://google.github.io/adk-docs/agents/)

In ADK, we create an **LLM Agent** by instantiating the `Agent` class with these key parameters:

- **name**: Unique identifier for the agent
- **description**: Brief explanation of the agent's purpose
- **model**: The LLM model to use (e.g., `gemini-2.5-flash`)
- **instruction**: The core prompt that defines the agent's behavior and personality
- **tools**: List of tools the agent can use to perform actions

---

### 🆕 NEW CONCEPT: Built-in Tools

> **What are Tools?**  
> Tools give agents capabilities to interact with the outside world. ADK provides **built-in tools** like `google_search` and code execution, as well as integrations with Google Cloud services and third-party APIs.
>
> 📖 **Reference**: [ADK Tools Documentation](https://google.github.io/adk-docs/tools/)

In this example, we give our agent the `google_search` tool so it can research trending topics on the web.

---

### What This Agent Does:
This agent is a **content strategist and topic researcher**. When you give it a niche or topic area, it will:
1. Use Google Search to find trending topics
2. Suggest 5-7 specific blog post titles
3. Explain why each topic is valuable for your audience

In [ ]:
from google.adk.agents import Agent
from google.adk.tools import google_search

def create_topic_research_agent():
    """Create a topic research agent for content creation"""
    return Agent(
        name="topic_research_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in researching trending blog topics and content ideas based on niche, audience, and keywords.",
        instruction="""
        You are an expert content strategist and topic researcher. Your goal is to help content creators
        find compelling blog topics using your search tool.

        You should:
        1.  **Understand the Niche:** If the user's request is vague, ask for details like target audience,
            content niche, and specific keywords or themes.
        2.  **Research Trends:** Use your search tool to find current trending topics, popular questions,
            and content gaps in the specified niche.
        3.  **Generate Ideas:** Suggest 5-7 specific blog post titles that are:
            - Relevant to the niche
            - Trending or evergreen
            - Actionable and specific
            - SEO-friendly
        4.  **Provide Context:** For each topic, briefly explain why it's a good choice (trend data,
            search volume insights, audience interest).
        5.  **Summarize:** Present the information in a clear, organized format with numbered suggestions.

        Be creative, data-driven, and always think about what will engage the target audience.
        """,
        tools=[google_search]  # 🔧 Giving the agent the google_search tool
    )

# Create the agent instance
topic_research_agent = create_topic_research_agent()
print(f"🧞 Agent '{topic_research_agent.name}' is created and ready!")

## 💾 4. Managing Conversation Memory with Sessions

### 🆕 NEW CONCEPT: Session

> **What is a Session?**  
> A Session represents a single, ongoing interaction between a user and an agent system. It "contains the chronological sequence of messages and actions taken by the agent (referred to as Events) during that specific interaction." Sessions also hold temporary data (State) relevant only during that particular conversation.
>
> 📖 **Reference**: [ADK Sessions Documentation](https://google.github.io/adk-docs/sessions/)

Think of a Session like a conversation thread. It remembers:
- All messages exchanged between user and agent
- Actions the agent has taken
- Temporary data relevant to this conversation

---

### 🆕 NEW CONCEPT: SessionService

> **What is SessionService?**  
> The SessionService "manages the complete lifecycle of conversation threads, including creating, retrieving, updating (by appending Events and modifying State), and deleting individual Sessions."
>
> 📖 **Reference**: [ADK Sessions Documentation](https://google.github.io/adk-docs/sessions/)

For this tutorial, we'll use **InMemorySessionService**, which stores conversation history in memory. This is perfect for:
- ✅ Local testing and development
- ✅ Fast prototyping
- ❌ NOT for production (data is lost when app restarts)

### Key Concepts:
- **Session**: Stores conversation history for context
- **SessionService**: Manages sessions (create, retrieve, store)
- **user_id**: Identifies the user across sessions

In [ ]:
from google.adk.sessions import InMemorySessionService

# Initialize our Session Service
session_service = InMemorySessionService()
my_user_id = "adk_content_creator_001"

print("✅ Session service initialized!")

## 🚀 5. Build the Execution Engine

### 🆕 NEW CONCEPT: Runner

> **What is the Runner?**  
> The Runner acts as the central orchestrator for agent invocations. It "receives the end user's query (new_message) and typically appends it to the session history via the SessionService," then coordinates execution by calling the agent's `run_async()` method and managing the event loop.
>
> 📖 **Reference**: [ADK Runtime Documentation](https://google.github.io/adk-docs/runtime/)

The **Runner** is the heart of the ADK runtime. When we call `runner.run_async()`, it
kicks off an **Event Loop**. This loop processes the user's message by invoking the
Execution Logic — which includes our agent's instructions, its LLM, and its tools.

### The Event-Driven Cycle:

```
1. User Query → Runner
2. Runner → Agent Execution Logic (LLM + Tools)
3. Execution Logic yields Events back to Runner
4. Runner processes events and updates state via SessionService
5. Execution Logic resumes with updated state
6. Repeat until final response
```

The `runner.run_async()` method gives us a **stream of events**. We listen
to this stream and wait for the `is_final_response()` event, which tells us the agent
has finished its work for that turn.

### Key Method: `run_async`

> **Why async?**  
> "The ADK Runtime is fundamentally built on asynchronous libraries" to handle LLM calls, tool executions, and other I/O operations concurrently without blocking. `run_async` is the recommended entry point for production applications.
>
> 📖 **Reference**: [ADK Runtime Documentation](https://google.github.io/adk-docs/runtime/)

In [ ]:
from IPython.display import display, Markdown
from google.adk.runners import Runner
from google.genai.types import Content, Part
from google.adk.sessions import Session

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    # Create the Runner - the execution orchestrator
    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        # run_async returns a stream of events
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            # Wait for the final response event
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

print("✅ Execution engine ready!")

## ✨ 6. Run Your First Agent!

Finally, we have our main `run_topic_research` function. Its job is to:
1. **Create a new Session** for our conversation using the session_service
2. **Define the user's query**
3. **Call `run_agent_query`** to kick off the entire process

When you run this code, the agent will:
- Receive your query
- Use Google Search to find trending topics and content ideas
- Formulate a final, organized list of blog topics with explanations

**Try modifying the query to match your own interests!**

In [ ]:
async def run_topic_research():
    # Create a new, single-use session for this query
    topic_research_session = await session_service.create_session(
        app_name=topic_research_agent.name,
        user_id=my_user_id
    )

    query = "I need blog topic ideas for a tech startup blog focused on AI and machine learning for small businesses"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(topic_research_agent, query, topic_research_session, my_user_id)

# Run the agent
await run_topic_research()

## 🎉 Recap: What We've Learned

To summarize what we learned today:

### Core Concepts Introduced:

1. **🆕 Agent** - A self-contained execution unit with instructions and tools  
   📖 [Agents Documentation](https://google.github.io/adk-docs/agents/)

2. **🆕 Tools (Built-in)** - Pre-built capabilities like `google_search`  
   📖 [Tools Documentation](https://google.github.io/adk-docs/tools/)

3. **🆕 Session** - Memory of a conversation containing message history  
   📖 [Sessions Documentation](https://google.github.io/adk-docs/sessions/)

4. **🆕 SessionService** - Manages session lifecycle (create, retrieve, update)  
   📖 [Sessions Documentation](https://google.github.io/adk-docs/sessions/)

5. **🆕 Runner** - The execution orchestrator that manages the event loop  
   📖 [Runtime Documentation](https://google.github.io/adk-docs/runtime/)

### The Execution Flow:
```
Agent Definition → Session Creation → Runner Execution → Event Loop → Final Response
```

---

## 🚀 What's Next?

Congratulations, you've successfully built and run your first AI agent! You've learned how
to define an agent with instructions, manage conversation state with a SessionService, and
execute it with the Runner.

In **Part 2**, we'll level up by teaching our agent new tricks. We'll move beyond built-in
tools and learn how to create our own **custom tools** (Python functions) to give our agent new, specialized
capabilities like SEO scoring, readability analysis, and hashtag generation.

### Preview of Part 2 Concepts:
- 🆕 Custom Function Tools
- 🆕 Tool Signatures and Docstrings
- 🆕 The Specialist Agent Pattern

See you in Part 2! 🚀